In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
epsilon = 1e-8
x = torch.randint(0,10,(5,), dtype=torch.float32)
y = torch.randint(0,10,(5,), dtype=torch.float32)
print(x.shape)
print(y.shape)
x = x.unsqueeze(0)
y = y.unsqueeze(1)
print(x.shape)
print(y.shape)
print(x)
print(y)
diff_matrix = torch.abs(x - y)
diff_matrix
# sim_matrix = 1.0 / diff_matrix

# sim_matrix = F.softmax(sim_matrix, dim=-1) / torch.sqrt(torch.tensor(y.shape[1]))

torch.Size([5])
torch.Size([5])
torch.Size([1, 5])
torch.Size([5, 1])
tensor([[2., 5., 4., 3., 6.]])
tensor([[7.],
        [7.],
        [8.],
        [4.],
        [7.]])


tensor([[5., 2., 3., 4., 1.],
        [5., 2., 3., 4., 1.],
        [6., 3., 4., 5., 2.],
        [2., 1., 0., 1., 2.],
        [5., 2., 3., 4., 1.]])

In [3]:
diff_matrix += epsilon
sim_matrix = 1.0 / diff_matrix
sim_matrix

tensor([[2.0000e-01, 5.0000e-01, 3.3333e-01, 2.5000e-01, 1.0000e+00],
        [2.0000e-01, 5.0000e-01, 3.3333e-01, 2.5000e-01, 1.0000e+00],
        [1.6667e-01, 3.3333e-01, 2.5000e-01, 2.0000e-01, 5.0000e-01],
        [5.0000e-01, 1.0000e+00, 1.0000e+08, 1.0000e+00, 5.0000e-01],
        [2.0000e-01, 5.0000e-01, 3.3333e-01, 2.5000e-01, 1.0000e+00]])

In [4]:
sim_matrix = F.softmax(sim_matrix, dim=-1)# / torch.sqrt(torch.tensor(y.shape[1]))
type(sim_matrix.size(-1))

int

In [5]:
num_heads = 2
dim = 3
bsz = 5
alphaq = torch.randint(10,(1,num_heads))
betaq = torch.randint(10,(1,num_heads))
alphak = torch.randint(10,(1,num_heads))
betak = torch.randint(10,(1,num_heads))
x = torch.randint(10,(bsz,dim,1))
print(x.shape)
print(alphaq.shape)
q = (x@alphaq + torch.ones((dim,1),dtype=torch.int64)@betaq).unsqueeze(1)
k = (x@alphak + torch.ones((dim,1), dtype=torch.int64)@betak).unsqueeze(2)
q[0], q[0].size(), k[0], k[0].size(), (q-k)[0], ((q-k)[0]).size()

torch.Size([5, 3, 1])
torch.Size([1, 2])


(tensor([[[38, 35],
          [23, 20],
          [23, 20]]]),
 torch.Size([1, 3, 2]),
 tensor([[[20, 37]],
 
         [[14, 22]],
 
         [[14, 22]]]),
 torch.Size([3, 1, 2]),
 tensor([[[ 18,  -2],
          [  3, -17],
          [  3, -17]],
 
         [[ 24,  13],
          [  9,  -2],
          [  9,  -2]],
 
         [[ 24,  13],
          [  9,  -2],
          [  9,  -2]]]),
 torch.Size([3, 3, 2]))

In [6]:
(q-k)[0][:,:,0]

tensor([[18,  3,  3],
        [24,  9,  9],
        [24,  9,  9]])

In [7]:
h=2
a_q = nn.Parameter(torch.Tensor(1,h))
x = torch.rand((3,1))
(x@a_q).size()

torch.Size([3, 2])

In [8]:
similarity = torch.randint(100,(10,3,3,2), dtype=torch.float32)
print(similarity[0][:,:,0])
similarity=F.softmax(similarity, dim=-2)
print(similarity[0][:,:,0])

tensor([[48.,  0., 23.],
        [50., 33., 89.],
        [14.,  7., 86.]])
tensor([[1.0000e+00, 1.4252e-21, 1.3888e-11],
        [1.1548e-17, 4.7809e-25, 1.0000e+00],
        [5.3802e-32, 4.9061e-35, 1.0000e+00]])


In [9]:
batch = 5
length = 3
heads =2
q = torch.randint(10, (batch,length,heads), dtype=torch.float32)
k = torch.randint(100,(batch,length,length,heads), dtype=torch.float32)
sim = torch.abs(q.unsqueeze(2)-k)
print(q[0,:,0])
print(k[0,:,:,0])
print(sim[0,:,:,0])
print(F.softmax(sim, dim=-2)[0,:,:,0])
print(q.unsqueeze(2).shape)
print(k.shape)
print(sim.shape)

tensor([2., 8., 2.])
tensor([[94., 59., 58.],
        [99., 32., 68.],
        [98., 18., 52.]])
tensor([[92., 57., 56.],
        [91., 24., 60.],
        [96., 16., 50.]])
tensor([[1.0000e+00, 6.3051e-16, 2.3195e-16],
        [1.0000e+00, 7.9849e-30, 3.4425e-14],
        [1.0000e+00, 1.8049e-35, 1.0531e-20]])
torch.Size([5, 3, 1, 2])
torch.Size([5, 3, 3, 2])
torch.Size([5, 3, 3, 2])


In [10]:
sim[0,0,:,0]

tensor([92., 57., 56.])

In [11]:
dim = 10
x = torch.rand(batch, dim)
query_weights = torch.randint(10,(dim,num_heads), dtype=torch.float32)

In [12]:
x.shape, query_weights.shape

(torch.Size([5, 10]), torch.Size([10, 2]))

In [13]:
diag_x = torch.stack([torch.diag(x[i]) for i in range(batch)])
q = torch.matmul(diag_x, query_weights)
q.shape

torch.Size([5, 10, 2])

In [14]:
query_biases = torch.randint(5,(dim, num_heads))
print(q[0])
print(query_biases)
full_q = q+query_biases
print(full_q[0])

tensor([[2.5160, 0.8387],
        [0.0000, 0.9487],
        [5.9575, 0.0000],
        [0.0000, 0.6820],
        [0.8469, 7.6225],
        [1.2656, 0.9492],
        [0.5763, 0.0000],
        [3.7921, 0.0000],
        [1.4309, 1.2264],
        [0.0000, 1.0502]])
tensor([[2, 0],
        [3, 3],
        [0, 0],
        [2, 1],
        [1, 2],
        [1, 4],
        [2, 4],
        [3, 1],
        [3, 1],
        [0, 0]])
tensor([[4.5160, 0.8387],
        [3.0000, 3.9487],
        [5.9575, 0.0000],
        [2.0000, 1.6820],
        [1.8469, 9.6225],
        [2.2656, 4.9492],
        [2.5763, 4.0000],
        [6.7921, 1.0000],
        [4.4309, 2.2264],
        [0.0000, 1.0502]])


In [16]:
k = torch.randint(10,(dim))

TypeError: randint(): argument 'size' (position 2) must be tuple of ints, not int

In [50]:
from importlib import reload
import fla_parallelized as a
reload(a)
x = torch.randint(10,(10,), dtype=torch.float32)
model = a.LinearSoftmax(10,5)
model(x)

tensor([4.7609, 4.4685, 4.8301, 4.6567, 4.9927], grad_fn=<ViewBackward0>)

In [51]:
model = a.MultiActivationLinear(input_dim=10, activations=[nn.ReLU, nn.Sigmoid, a.GaussianActivation] ,nodes_per_activation=2)
x = torch.randint(10,(10,10), dtype=torch.float32)


(tensor([[9., 3., 3., 7., 8., 7., 4., 3., 0., 2.],
         [6., 1., 2., 9., 9., 3., 8., 2., 4., 2.],
         [4., 3., 1., 7., 6., 4., 1., 3., 1., 3.],
         [9., 1., 6., 7., 3., 4., 1., 7., 5., 4.],
         [1., 4., 8., 6., 1., 1., 3., 4., 7., 8.],
         [8., 3., 8., 3., 6., 8., 7., 2., 1., 8.],
         [9., 7., 2., 6., 2., 2., 7., 2., 7., 4.],
         [3., 0., 0., 2., 1., 9., 5., 4., 5., 7.],
         [4., 1., 2., 1., 9., 8., 4., 6., 7., 0.],
         [7., 4., 5., 0., 4., 5., 2., 6., 7., 3.]]),
 tensor([[0.0000, 0.7746],
         [0.0000, 0.0000],
         [0.0000, 0.0000],
         [0.7184, 0.6146],
         [1.5866, 0.0000],
         [1.2549, 2.4687],
         [0.0000, 0.0000],
         [0.0000, 0.0000],
         [0.4834, 0.0000],
         [0.3534, 0.0000],
         [0.5376, 0.6380],
         [0.2101, 0.6911],
         [0.4776, 0.2762],
         [0.6951, 0.5044],
         [0.1637, 0.4765],
         [0.2528, 0.8749],
         [0.4655, 0.7006],
         [0.7071, 0.2550],
  

In [106]:
def sim(row, ep=1e-8, a=1, b=0.09):
    return [1/(abs(a)*abs(x+b)+ep) for x in row]
F.softmax(torch.tensor(sim([0.001,2,3,4,5,6,7]), dtype=torch.float32))

/var/folders/dz/88r06kr52bj9mbdq8tdjz0m80000gn/T/ipykernel_33576/892072556.py:3: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  F.softmax(torch.tensor(sim([0.001,2,3,4,5,6,7]), dtype=torch.float32))


tensor([9.9987e-01, 2.7244e-05, 2.3336e-05, 2.1561e-05, 2.0549e-05, 1.9897e-05,
        1.9442e-05])

In [115]:
x = torch.randint(10,(10,10), dtype=torch.float32)
x.softmax(0)

tensor([[4.6153e-01, 2.3733e-01, 2.9955e-02, 4.1295e-01, 1.5652e-03, 1.2752e-02,
         7.4756e-03, 9.7862e-04, 3.5584e-04, 9.8035e-05],
        [1.5483e-04, 1.1816e-02, 2.2134e-01, 2.0559e-02, 2.3229e-01, 9.4227e-02,
         5.5238e-02, 1.3244e-04, 5.2812e-02, 2.9224e-01],
        [3.1097e-03, 3.2119e-02, 1.4914e-03, 5.5886e-02, 3.1437e-02, 4.6913e-03,
         1.5015e-01, 5.3431e-02, 4.8158e-05, 2.9224e-01],
        [5.6957e-05, 2.3733e-01, 2.2134e-01, 1.3853e-04, 2.3229e-01, 6.3489e-04,
         1.5015e-01, 3.9480e-01, 3.9023e-01, 3.6065e-05],
        [6.2461e-02, 5.8828e-04, 1.4914e-03, 4.1295e-01, 1.5652e-03, 6.3489e-04,
         4.0815e-01, 3.6001e-04, 3.5584e-04, 7.2439e-04],
        [4.2086e-04, 2.3733e-01, 2.0184e-04, 3.7656e-04, 4.2546e-03, 9.4227e-02,
         3.7219e-04, 3.6001e-04, 1.9428e-02, 1.4550e-02],
        [1.1440e-03, 2.3733e-01, 2.2134e-01, 2.0559e-02, 3.1437e-02, 1.7258e-03,
         5.5238e-02, 3.9480e-01, 3.9023e-01, 2.9224e-01],
        [1.1440e-03, 2.1642

In [123]:
b=10
d = 5
h=3
slice = 1
similarity_matrix = torch.randint(2,(b,d,d,h))
alphas = torch.randint(10,(h,))
print(similarity_matrix[0,:,:,slice])
print(alphas)
print((similarity_matrix*alphas)[0,:,:,slice])

tensor([[1, 1, 1, 0, 1],
        [1, 0, 0, 1, 0],
        [1, 0, 1, 0, 0],
        [1, 1, 1, 0, 0],
        [1, 0, 0, 0, 1]])
tensor([1, 6, 7])
tensor([[6, 6, 6, 0, 6],
        [6, 0, 0, 6, 0],
        [6, 0, 6, 0, 0],
        [6, 6, 6, 0, 0],
        [6, 0, 0, 0, 6]])


In [18]:
import time
def compute_S_multiheaded(x, y, epsilon=1e-8):
    x = x.unsqueeze(2)
    y = y.unsqueeze(1)
    
    abs_diff = torch.abs(x - y) + epsilon
    reciprocal_diff = 1 / abs_diff
    
    sum_reciprocal_diff = reciprocal_diff.sum(dim=2, keepdim=True)
    reciprocal_diff /= (sum_reciprocal_diff * torch.sqrt(torch.tensor(x.shape[-1], dtype=reciprocal_diff.dtype)))
    
    return reciprocal_diff



def compute_sim(x, y, epsilon=1e-8):
    x = x.unsqueeze(1)
    y = y.unsqueeze(2)
    diff_matrix = torch.abs(x - y) + epsilon
    # diff_matrix += epsilon
    sim_matrix = torch.reciprocal(diff_matrix)
    sim_matrix = F.softmax(sim_matrix, dim=-2) / torch.sqrt(torch.tensor(sim_matrix.shape[-1]))
    return sim_matrix

# Example usage
batch_size = 128
n = 400
h=30

x = torch.randint(10, (batch_size,n,h))
y = torch.randint(10, (batch_size,n,h))

start = time.time()
compute_S_multiheaded(x, y)
end = time.time()
print(f'non softmax time: {end-start}')

start = time.time()
compute_sim(x, y)
end = time.time()
print(f'softmax time: {end-start}')


non softmax time: 3.479398727416992
softmax time: 4.029902219772339
